### Python Project ETL - AirBnB Listing 2024

**steps**

1. Importing all dependencies (lib)
2. Extract datasets
3. Transform/Clean Datasets
4. Load Dataset into Postgres

### Task 1: Importing All Dependencies

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg
from datetime import datetime

%matplotlib inline

### Task 2: Extracting Datasets

In [40]:
def extract_csv_to_dataframe(filepath):
    try:
        df = pd.read_csv(filepath)
        return df
    except FileNotFoundError:
        print(f"Error: the file '{filepath}' was not found.")
        return None
    except Exception as e:
        print(f"An error occured {e}")
        return None
    
file_path = 'datasets.csv'
data = extract_csv_to_dataframe(file_path)

if data is not None:
    print("CSV data extracted successfully into a dataframe:")
    print(data.head())

CSV data extracted successfully into a dataframe:
             id                                               name    host_id  \
0  1.312228e+06         Rental unit in Brooklyn · ★5.0 · 1 bedroom    7130382   
1  4.527754e+07  Rental unit in New York · ★4.67 · 2 bedrooms ·...   51501835   
2  9.710000e+17  Rental unit in New York · ★4.17 · 1 bedroom · ...  528871354   
3  3.857863e+06  Rental unit in New York · ★4.64 · 1 bedroom · ...   19902271   
4  4.089661e+07  Condo in New York · ★4.91 · Studio · 1 bed · 1...   61391963   

            host_name neighbourhood_group       neighbourhood   latitude  \
0              Walter            Brooklyn        Clinton Hill  40.683710   
1            Jeniffer           Manhattan      Hell's Kitchen  40.766610   
2              Joshua           Manhattan             Chelsea  40.750764   
3  John And Catherine           Manhattan  Washington Heights  40.835600   
4      Stay With Vibe           Manhattan         Murray Hill  40.751120   

   lon

### Transforming Datasets

In [35]:
from xml.etree.ElementInclude import include


def remove_na(data, strategy='mean'):
    """
    Handles missing values in numerical columns using a specified strategy 
    """
    for col in data.select_dtypes(include=['number']).columns:
        if data[col].isnull().any():
            if strategy == 'mean':
                data[col].fillna(data[col].mean(), inplace=True)
            elif strategy == 'median':
                data[col].fillna(data[col].median(), inplace=True)
            elif strategy == 'mode':
                data[col].fillna(data[col].mode(), inplace=True)
            elif strategy == 'ffill': # Forward fill for non-numerical
                data[col].fillna(method='ffill', inplace=True)
    return data



### Task 3: Initial Exploration

In [42]:
data.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,rating,bedrooms,beds,baths
0,1.312228e+06,Rental unit in Brooklyn · ★5.0 · 1 bedroom,7130382,Walter,Brooklyn,Clinton Hill,40.683710,-73.964610,Private room,55.0,...,20/12/15,0.03,1.0,0.0,0.0,No License,5,1,1,Not specified
1,4.527754e+07,Rental unit in New York · ★4.67 · 2 bedrooms ·...,51501835,Jeniffer,Manhattan,Hell's Kitchen,40.766610,-73.988100,Entire home/apt,144.0,...,01/05/23,0.24,139.0,364.0,2.0,No License,4.67,2,1,1
2,9.710000e+17,Rental unit in New York · ★4.17 · 1 bedroom · ...,528871354,Joshua,Manhattan,Chelsea,40.750764,-73.994605,Entire home/apt,187.0,...,18/12/23,1.67,1.0,343.0,6.0,Exempt,4.17,1,2,1
3,3.857863e+06,Rental unit in New York · ★4.64 · 1 bedroom · ...,19902271,John And Catherine,Manhattan,Washington Heights,40.835600,-73.942500,Private room,120.0,...,17/09/23,1.38,2.0,363.0,12.0,No License,4.64,1,1,1
4,4.089661e+07,Condo in New York · ★4.91 · Studio · 1 bed · 1...,61391963,Stay With Vibe,Manhattan,Murray Hill,40.751120,-73.978600,Entire home/apt,85.0,...,03/12/23,0.24,133.0,335.0,3.0,No License,4.91,Studio,1,1


In [43]:
data.shape

(20770, 22)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20770 entries, 0 to 20769
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20770 non-null  float64
 1   name                            20770 non-null  object 
 2   host_id                         20770 non-null  int64  
 3   host_name                       20770 non-null  object 
 4   neighbourhood_group             20770 non-null  object 
 5   neighbourhood                   20763 non-null  object 
 6   latitude                        20770 non-null  float64
 7   longitude                       20770 non-null  float64
 8   room_type                       20763 non-null  object 
 9   price                           20770 non-null  float64
 10  minimum_nights                  20770 non-null  float64
 11  number_of_reviews               20770 non-null  float64
 12  last_review                     

In [45]:
#statistical summmary
data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,beds
count,2.077000e+04,2.077000e+04,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000,20770.000000
mean,3.033858e+17,1.749049e+08,40.726821,-73.939179,187.714940,28.558493,42.610605,1.257589,18.866686,206.067957,10.848962,1.723592
std,3.901221e+17,1.725657e+08,0.060283,0.061392,1022.407226,33.527045,73.511010,1.904151,70.909491,135.054493,21.351277,1.211993
min,2.595000e+03,1.678000e+03,40.500314,-74.249840,10.000000,1.000000,1.000000,0.010000,1.000000,0.000000,0.000000,1.000000
25%,2.707260e+07,2.041184e+07,40.684173,-73.980741,80.000000,30.000000,4.000000,0.210000,1.000000,87.000000,1.000000,1.000000
50%,4.992852e+07,1.086990e+08,40.722937,-73.949587,125.000000,30.000000,14.000000,0.650000,2.000000,215.000000,3.000000,1.000000
75%,7.220000e+17,3.143997e+08,40.763100,-73.917490,199.000000,30.000000,49.000000,1.800000,5.000000,353.000000,15.000000,2.000000
max,1.050000e+18,5.504035e+08,40.911147,-73.713650,100000.000000,1250.000000,1865.000000,75.490000,713.000000,365.000000,1075.000000,42.000000


### Task 4: Data Cleaning

In [46]:
data.isnull().sum()

# dropping all missing values
data.dropna(inplace=True)

# data.fillna()

In [48]:
data.duplicated().sum()

# deleting all duplicated rows
#data[data.duplicated()]

data.drop_duplicates(inplace=True)
data.duplicated().sum()

np.int64(0)

In [77]:
# Drop Columns
data.drop(columns='baths', inplace=True)
data.drop(columns='rating', inplace=True)

In [64]:
#Change date format
data['last_review'] = pd.to_datetime(data['last_review'], format='%d/%m/%y')
data.dtypes

id                                        object
name                                      object
host_id                                   object
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                    float64
minimum_nights                           float64
number_of_reviews                        float64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count           float64
availability_365                         float64
number_of_reviews_ltm                    float64
license                                   object
rating                                    object
bedrooms                                  object
beds                

In [49]:
# type casting
# changing data types

data.dtypes

data['id'] = data['id'].astype(object)
data.dtypes

data['host_id'] = data['host_id'].astype(object)
data.dtypes

id                                 object
name                               object
host_id                            object
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                             float64
minimum_nights                    float64
number_of_reviews                 float64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count    float64
availability_365                  float64
number_of_reviews_ltm             float64
license                            object
rating                             object
bedrooms                           object
beds                                int64
baths                              object
dtype: object

### Load into Database

In [80]:
# Connect to PostgreSQL database
conn = psycopg.connect (dbname='airbnb_db',
user='postgres',
password='hasibjan1',
host='localhost',
port="5432")

with conn:
    with conn.cursor() as cur:

        # Drop table if exists
        cur.execute(
         "DROP TABLE IF EXISTS house_data")
        
        #Create table for database
        cur.execute(
        """
        CREATE TABLE IF NOT EXISTS house_data (
        id                                 BIGINT,
        name                               VARCHAR(100),
        host_id                            BIGINT,
        host_name                          VARCHAR(100),
        neighbourhood_group                VARCHAR(50),
        neighbourhood                      VARCHAR(50),
        latitude                           FLOAT,
        longitude                          FLOAT,
        room_type                          VARCHAR(50),
        price                              INTEGER,
        minimum_nights                     INTEGER,
        number_of_reviews                  INTEGER,
        last_review                        DATE,
        reviews_per_month                  FLOAT,
        calculated_host_listings_count     INTEGER,
        availability_365                   INTEGER,
        number_of_reviews_ltm              INTEGER,
        license                            VARCHAR(50),
        bedrooms                           VARCHAR(50),
        beds                               INTEGER
        )
        """
        )


        # Insert data 
        for row in data.itertuples(index=False):
          cur.execute(
          """
          INSERT INTO house_data (
          id,
          name,
          host_id,
          host_name,
          neighbourhood_group,
          neighbourhood,
          latitude,
          longitude,
          room_type,
          price,
          minimum_nights,
          number_of_reviews,
          last_review,
          reviews_per_month,
          calculated_host_listings_count,
          availability_365,
          number_of_reviews_ltm,
          license,
          bedrooms,
          beds
          )
          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
          """, row

          )

    